In [37]:
import requests

API_KEY = "916a0cea5b90fb3ae7290f9a58c361fd"
URL = f"https://api.the-odds-api.com/v4/sports/americanfootball_nfl/odds?regions=us&oddsFormat=american&apiKey={API_KEY}"

response = requests.get(URL)
odds_data = response.json()

def calculate_arbitrage(odds):
    return sum(1/odd for odd in odds if odd > 0) < 1

for match in odds_data:
    match_id = match['id']
    sport_title = match['sport_title']
    home_team = match['home_team']
    away_team = match['away_team']
    
    bookmakers = match['bookmakers']
    for bookmaker in bookmakers:
        bookmaker_title = bookmaker['title']
        markets = bookmaker['markets']
        
        for market in markets:
            market_key = market['key']
            outcomes = market['outcomes']
            odds = [outcome['price'] for outcome in outcomes]
            
            if calculate_arbitrage(odds):
                print(f"Arbitrage opportunity in match: {sport_title} - {home_team} vs {away_team}")
                print(f"Bookmaker: {bookmaker_title}, Market: {market_key}, Odds: {odds}")

def implied_probability(odd):
    if odd > 0:
        return 1 - (odd / (odd + 100))
    else:
        return abs(odd) / (abs(odd) + 100)

for match in odds_data:
    sport_title = match['sport_title']
    home_team = match['home_team']
    away_team = match['away_team']
    
    print(f"Match: {home_team} vs {away_team}")
    
    bookmakers = match['bookmakers']
    for bookmaker in bookmakers:
        bookmaker_title = bookmaker['title']
        markets = bookmaker['markets']
        
        for market in markets:
            market_key = market['key']
            if market_key == 'h2h':  # Only consider head-to-head (moneyline) odds
                outcomes = market['outcomes']
                
                team_probabilities = {}
                for outcome in outcomes:
                    team_name = outcome['name']
                    price = outcome['price']
                    probability = implied_probability(price)
                    team_probabilities[team_name] = probability
                
                print(f"  Bookmaker: {bookmaker_title}")
                for team, prob in team_probabilities.items():
                    print(f"    {team}: Implied Probability = {prob:.2%}")


Arbitrage opportunity in match: NFL - Kansas City Chiefs vs Baltimore Ravens
Bookmaker: DraftKings, Market: h2h, Odds: [130, -155]
Arbitrage opportunity in match: NFL - Kansas City Chiefs vs Baltimore Ravens
Bookmaker: FanDuel, Market: h2h, Odds: [128, -152]
Arbitrage opportunity in match: NFL - Kansas City Chiefs vs Baltimore Ravens
Bookmaker: Caesars, Market: h2h, Odds: [130, -155]
Arbitrage opportunity in match: NFL - Kansas City Chiefs vs Baltimore Ravens
Bookmaker: BetOnline.ag, Market: h2h, Odds: [127, -147]
Arbitrage opportunity in match: NFL - Kansas City Chiefs vs Baltimore Ravens
Bookmaker: LowVig.ag, Market: h2h, Odds: [127, -147]
Arbitrage opportunity in match: NFL - Kansas City Chiefs vs Baltimore Ravens
Bookmaker: BetRivers, Market: h2h, Odds: [125, -152]
Arbitrage opportunity in match: NFL - Kansas City Chiefs vs Baltimore Ravens
Bookmaker: BetMGM, Market: h2h, Odds: [125, -150]
Arbitrage opportunity in match: NFL - Kansas City Chiefs vs Baltimore Ravens
Bookmaker: MyBoo

SyntaxError: invalid syntax (1492990763.py, line 1)

In [43]:
import requests

API_KEY = "916a0cea5b90fb3ae7290f9a58c361fd"
URL = f"https://api.the-odds-api.com/v4/sports/americanfootball_nfl/odds?regions=us&oddsFormat=american&apiKey={API_KEY}"

response = requests.get(URL)
odds_data = response.json()

def implied_probability(odd):
    if odd > 0:
        return 1 - (odd / (odd + 100))
    else:
        return abs(odd) / (abs(odd) + 100)

arbitrage_count = 0
no_arbitrage_count = 0

for match in odds_data:
    sport_title = match['sport_title']
    home_team = match['home_team']
    away_team = match['away_team']
    
    print(f"Checking match: {home_team} vs {away_team}")
    
    bookmakers = match['bookmakers']
    
    best_opportunity = None
    for i, bookmaker1 in enumerate(bookmakers):
        for j, bookmaker2 in enumerate(bookmakers):
            if i != j:
                # Get outcomes for both bookmakers
                outcomes1 = bookmaker1['markets'][0]['outcomes']
                outcomes2 = bookmaker2['markets'][0]['outcomes']
                
                # Calculate probabilities for all combinations
                prob_home_book1 = implied_probability(outcomes1[0]['price'])
                prob_away_book2 = implied_probability(outcomes2[1]['price'])
                combined_probability1 = prob_home_book1 + prob_away_book2
                
                prob_away_book1 = implied_probability(outcomes1[1]['price'])
                prob_home_book2 = implied_probability(outcomes2[0]['price'])
                combined_probability2 = prob_away_book1 + prob_home_book2
                
                # Check if either combined probability is less than 100%
                if combined_probability1 < 1:
                    best_opportunity = {
                        "home_team": home_team,
                        "away_team": away_team,
                        "book1": bookmaker1['title'],
                        "book2": bookmaker2['title'],
                        "combined_probability": combined_probability1,
                        "home_team_odds": outcomes1[0]['price'],
                        "away_team_odds": outcomes2[1]['price']
                    }
                if combined_probability2 < 1:
                    best_opportunity = {
                        "home_team": away_team,  # Teams are swapped here
                        "away_team": home_team,
                        "book1": bookmaker1['title'],
                        "book2": bookmaker2['title'],
                        "combined_probability": combined_probability2,
                        "home_team_odds": outcomes1[1]['price'],
                        "away_team_odds": outcomes2[0]['price']
                    }
    
    # If a good opportunity is found, print the details and increment arbitrage_count
    if best_opportunity:
        print(f"Arbitrage opportunity found in match: {best_opportunity['home_team']} vs {best_opportunity['away_team']}")
        print(f"  Bet on {best_opportunity['home_team']} with {best_opportunity['book1']} at odds {best_opportunity['home_team_odds']}")
        print(f"  Bet on {best_opportunity['away_team']} with {best_opportunity['book2']} at odds {best_opportunity['away_team_odds']}")
        print(f"  Combined Probability: {best_opportunity['combined_probability']:.2%}")
        arbitrage_count += 1
    else:
        print("No arbitrage opportunity found.")
        no_arbitrage_count += 1

# Tally the results
print(f"\nTotal Arbitrage Opportunities Found: {arbitrage_count}")
print(f"Total Matches with No Arbitrage Opportunity: {no_arbitrage_count}")


Checking match: Kansas City Chiefs vs Baltimore Ravens
No arbitrage opportunity found.
Checking match: Philadelphia Eagles vs Green Bay Packers
No arbitrage opportunity found.
Checking match: Buffalo Bills vs Arizona Cardinals
No arbitrage opportunity found.
Checking match: Atlanta Falcons vs Pittsburgh Steelers
No arbitrage opportunity found.
Checking match: New Orleans Saints vs Carolina Panthers
No arbitrage opportunity found.
Checking match: Chicago Bears vs Tennessee Titans
No arbitrage opportunity found.
Checking match: Cincinnati Bengals vs New England Patriots
No arbitrage opportunity found.
Checking match: Indianapolis Colts vs Houston Texans
No arbitrage opportunity found.
Checking match: Miami Dolphins vs Jacksonville Jaguars
No arbitrage opportunity found.
Checking match: New York Giants vs Minnesota Vikings
No arbitrage opportunity found.
Checking match: Seattle Seahawks vs Denver Broncos
No arbitrage opportunity found.
Checking match: Los Angeles Chargers vs Las Vegas Rai

In [71]:
import requests

API_KEY = "YOUR_API_KEY"
EVENT_ID = "a512a48a58c4329048174217b2cc7ce0"  # Example event ID for an NFL game
REGIONS = "us"
ODDS_FORMAT = "american"
MARKET_KEY = "player_pass_yds"  # Market for Player Passing Yards

# Function to get all available bets for player_pass_yds
def get_all_player_passing_yards_bets(event_id, market_key):x`
    URL = f"https://api.the-odds-api.com/v4/sports/americanfootball_nfl/events/{event_id}/odds"
    params = {
        "apiKey": API_KEY,
        "regions": REGIONS,
        "markets": market_key,
        "oddsFormat": ODDS_FORMAT
    }
    response = requests.get(URL, params=params)
    odds_data = response.json()

    # Check if 'bookmakers' is in the response and not empty
    if 'bookmakers' not in odds_data or not odds_data['bookmakers']:
        print("No bookmakers found for this event.")
        return

    # Print all available bets for player_pass_yds
    for bookmaker in odds_data['bookmakers']:
        print(f"\nBookmaker: {bookmaker['title']}")
        for market in bookmaker['markets']:
            if market['key'] == market_key:
                for outcome in market['outcomes']:
                    print(f"Player: {outcome['name']}")
                    print(f"Outcome: {outcome['price']}")
                    print(f"Last Update: {market['last_update']}\n")

# Main function to open all bets on player_pass_yds
def main():
    get_all_player_passing_yards_bets(EVENT_ID, MARKET_KEY)

if __name__ == "__main__":
    main()


No bookmakers found for this event.
